In [ ]:
# Standard library imports
import os
import re
import sys
import xml.etree.ElementTree as ET
from collections import defaultdict

# Third-party library imports
from tabulate import tabulate

# GUI-related imports
import tkinter as tk
import tkinter.scrolledtext as st
from tkinter import filedialog, messagebox, ttk


def select_folder(var):
    """ Function to select a folder """
    folder = filedialog.askdirectory()
    if folder:
        var.set(folder)


def get_all_files(input_folder, output_folder):
    """ returns a list of tuples of in and out files"""
    
    allFiles = []
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.eaf'):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, f"{os.path.splitext(file_name)[0]}.eaf")
            allFiles.append([input_path, output_path])
    
    return allFiles


def get_data(file_path):
    """ gets all REF_ANNOTATIONS out of the ELAN file
    """
    
    data = {} # get all annotations into a standard format

    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Iterate over each TIER element
    for tier in root.findall("TIER"):
        tier_type = tier.get("LINGUISTIC_TYPE_REF", "")

        # Iterate over each REF_ANNOTATION element
        for nummer, ref_annotation in enumerate(tier.findall(".//REF_ANNOTATION")):
            annotation_id = ref_annotation.get("ANNOTATION_ID", "")
            annotation_ref = ref_annotation.get("ANNOTATION_REF", "")
            annotation_value = ref_annotation.find("ANNOTATION_VALUE").text if ref_annotation.find("ANNOTATION_VALUE") is not None else ""

            # Append the extracted data as a dictionary
            data[annotation_id] = {
                #"File": file_name,
                "TIER_TYPE": tier_type,
                "ANNOTATION_ID": annotation_id,
                "ANNOTATION_REF": annotation_ref,
                "ANNOTATION_VALUE": annotation_value,
                "ANNOTATION_NUMBER": nummer,
                "CHILDREN": []
            }
    return data

    
def find_children(data):
    
    """ find all children of a annotation
        finds other annotations which ref this one"""
    for k, v in data.items():
        parent = v["ANNOTATION_REF"]
        if parent in data:
            data[parent]["CHILDREN"].append(k)
            
    return data


def clean_pattern(pattern):
    """ if there is no key given, ".*" is default
    """
    if list(pattern.keys())[0] == '':
        search_type = ".*"
    else:
        search_type = list(pattern.keys())[0]
        
    search_string = list(pattern.values())[0]
    
    return search_type, search_string


def add_to_stats(v, stats, data):
    """add to statistics """
    
    if v['ANNOTATION_VALUE'] not in stats:
        stats[v['ANNOTATION_VALUE']] = {'tier': {}, 'parents': {}, 'children': {}, "count": 0}

    if v["TIER_TYPE"] not in stats[v['ANNOTATION_VALUE']]["tier"]:
        stats[v['ANNOTATION_VALUE']]["tier"][v["TIER_TYPE"]] = 0

    stats[v['ANNOTATION_VALUE']]["tier"][v["TIER_TYPE"]] += 1

    stats[v['ANNOTATION_VALUE']]["count"] += 1
    
    return stats


def grouping_stats(source):
    """ creates what will be added into a single cell in the dispay table"""
    cell_content = []
    for item, nr in source.items():
            cell_content.append(item + " " + str(nr))
    
    return "\n".join(cell_content)
    
    
def print_stats(stats):
    """ construct and print the stats table"""
    
    stats_print = [["term", "tier", "parent", 'children']] # table header
    
    for k, v in stats.items():
        
        term = k + " " + str(v["count"])
        
        tiers = grouping_stats(v['tier'])
        
        eltern = grouping_stats(v['parents'])

        kinder = grouping_stats(v['children'])

        stats_print.append([term, tiers, eltern, kinder])

    print ("\n\n  STATISTICS FOR ALL FILES: ", )
    print(tabulate(stats_print, headers='firstrow', tablefmt='fancy_grid')) 

    
def add_item_to_stats(v, stats, item, group):
    """ for display in stats"""
            
    if item not in stats[v['ANNOTATION_VALUE']][group]:

        stats[v['ANNOTATION_VALUE']][group][item] = 0

    stats[v['ANNOTATION_VALUE']][group][item] +=1
    
    return stats


def assemble_tier_value(data, relative):
    """for display in table"""
    return data[relative]["TIER_TYPE"] + ": " + data[relative]["ANNOTATION_VALUE"]

    
def add_to_file_table(v, stats, file_table, data):
                        
    parent = ""

    stats = add_to_stats(v, stats, data)

    target = v["TIER_TYPE"] + ": " + v["ANNOTATION_VALUE"]

    #get parents
    if v["ANNOTATION_REF"] in data:
        papa = v["ANNOTATION_REF"]
        parent = assemble_tier_value(data, papa) 
        stats = add_item_to_stats(v, stats, parent, "parents")

    # get children
    children = []

    if len(v["CHILDREN"]) > 0:

        for child in v["CHILDREN"]:
            kid = assemble_tier_value(data, child)
            stats = add_item_to_stats(v, stats, kid, "children")
            
            children.append(kid)

    file_table.append([target, parent, "\n".join(children)]) 
    
    return file_table, stats


def check_if_match(v, search_type, search_string):
    """ check if search_type and search_string match"""
    # no string no results
    if v['ANNOTATION_VALUE']:

        # filter for TIER_TYPE
        if re.fullmatch(search_type, v['TIER_TYPE']):

            # find the string or regex
            if re.fullmatch(search_string, v['ANNOTATION_VALUE']):
                return True
            
    return False


def find_string(input_folder, pattern):
    
    search_type, search_string = clean_pattern(pattern)
    
    output_text.delete("1.0", "end")
    
    stats ={} #dict for the statistical info accross all files
    
    input_folder = input_path_var.get()
    output_folder = output_folder_var.get()
    
    all_files = get_all_files(input_folder, output_folder)

    for file_in, file_out in all_files:

        print ("\n", " ", file_in, "\n")

        file_table = [['term', 'parent', 'children']]
        
        data = get_data(file_in)
        
        data = find_children(data)
        
        for k, v in data.items():
            
            match_check = check_if_match(v, search_type, search_string)
            
            if match_check:
                file_table, stats = add_to_file_table(v, stats, file_table, data)
                
        print(tabulate(file_table, headers='firstrow', tablefmt='fancy_grid'))   
    
    print_stats(stats)
    
    
    output_text.see("1.0")

    
def find_match(anno):
    """ get all family members of a annotation into a list, itself, it's children, it's parents"""
    anno_id = anno["ANNOTATION_ID"]
    anno_ref = anno["ANNOTATION_REF"]
    anno_children = anno["CHILDREN"]
    anno_list = [anno["ANNOTATION_ID"], anno["ANNOTATION_REF"]]
    for x in anno_children:
        anno_list.append(x)
    return anno_list



def two_bottom_rows_filled(match2, match3):
    """ the bottom two rows are loaded"""
    change_list = []
    partners = []
    
    for third in match3:
        list3 = find_match(third)
        
        for second in match2:
            list2 = find_match(second)

            common_all = set(list2) & set(list3)
                        
            if common_all:
                partners.append([second,third])
                change_list.append(third["ANNOTATION_ID"])
                
    return change_list, partners


def one_rows_filled(match3):
    """ Only the last row is filled with terms
    """
    change_list = []
    partners = []
    for third in match3:
        
        list3 = find_match(third)

        common_all = set(list3)

        # Results
        if common_all:
            #print(f"All lists share at least one ID: {common_all}")
            partners.append([third])
            change_list.append(third["ANNOTATION_ID"])

    return change_list, partners


def write_to_file(change_list, file_path, output_path, old, new):
    #for each in matches:
    #    print (each)

    # replace all matching annotations based on the replacement dictionary
    tree = ET.parse(file_path)

    root = tree.getroot()

    # Iterate over each TIER element
    for tier in root.findall("TIER"):
        tier_type = tier.get("LINGUISTIC_TYPE_REF", "")

        # Iterate over each REF_ANNOTATION element
        for ref_annotation in tier.findall(".//REF_ANNOTATION"):
            annotation_id = ref_annotation.get("ANNOTATION_ID", "")
            annotation_value = ref_annotation.find("ANNOTATION_VALUE").text if ref_annotation.find("ANNOTATION_VALUE") is not None else ""

            infos = []

            if annotation_id in change_list:
                
                ref_annotation.find("ANNOTATION_VALUE").text = new

                print("   ", ", ".join(infos), ":: ", tier_type, annotation_id, annotation_value, " --> ",new)

    tree.write(output_path, encoding="utf-8", xml_declaration=True)
    
    output_text.see("1.0")


def get_all_anno_IDs(root):
    """ get a list of all already used ANNOTATION_IDs"""
    all_anno_IDs = {
        int(anno.get("ANNOTATION_ID", "")[1:])
        for tier in root.findall("TIER")
        for anno in tier.findall(".//ALIGNABLE_ANNOTATION") + tier.findall(".//REF_ANNOTATION")
    }
    return sorted(all_anno_IDs)


def print_child_list(change_list, data):
    children_list = []
    for k, v in data.items():
        if v["ANNOTATION_REF"] in change_list:
            if v["TIER_TYPE"] not in children_list:
                children_list.append(v["TIER_TYPE"])
    
    return children_list


def generate_split_pattern(key3, word3, replacer3, 
                           key4, replacer4,
                           key5, replacer5,
                           key6, replacer6):

    split_pattern = {key3: replacer3, key4: replacer4, key5: replacer5, key6: replacer6}
    #print (split_pattern)
    return(split_pattern)

    
def get_anno_info(ref_annotation):
    """ get all info of the annotation"""
    annotation_id = ref_annotation.get("ANNOTATION_ID", "")
    annotation_ref = ref_annotation.get("ANNOTATION_REF", "")
    annotation_previous = ref_annotation.get("PREVIOUS_ANNOTATION", "")
    annotation_value = ref_annotation.find("ANNOTATION_VALUE").text if ref_annotation.find("ANNOTATION_VALUE") is not None else ""
    return annotation_id, annotation_ref, annotation_previous, annotation_value, annotation_previous


def add_split_pattern(ref_annotation, last_anno, tier, split_pattern):
    """add the first 'main' of the split patterns"""
    annotation_id, annotation_ref, _, annotation_value, annotation_previous = get_anno_info(ref_annotation)
    anno_ids = []
    
    
    
    for i, new_value in enumerate(split_pattern):
        last_anno += 1
        anno_id = f'a{last_anno}'

        # Create new annotation
        new_annotation_element, _ = make_anno(tier, 
                                              last_anno, 
                                              anno_id, 
                                              annotation_ref, 
                                              annotation_id, 
                                              new_value)

        remove_first_previous_anno(new_annotation_element, i)
        
        # in case the original annotation had a PREVIOUS_ANNOTATION, this info is added to the first new annotation
        if annotation_previous:
            if i == 0:
                new_annotation_element[0].attrib["PREVIOUS_ANNOTATION"] = annotation_previous

        anno_ids.append(anno_id)
    
    #print (anno_ids)
    print(f"{annotation_value} --> {'|'.join(split_pattern)}")
    return last_anno, anno_ids


def get_split_pattern(tier_type, split_pattern_all):
    """ returns the split pattern for the tier and X if not found"""
    return split_pattern_all.get(tier_type, "X|X|X|X|X|X|X|X|X|X|X|X|X|X").split('|')


def make_anno(tier, last_anno, anno_id, anno_ref, annotation_id, anno_value):
    # Create new annotation to add to tier
    new_annotation_element = ET.SubElement(tier, "ANNOTATION")

    alignable_annotation = ET.SubElement(
        new_annotation_element,
        "REF_ANNOTATION",
        {
            "ANNOTATION_ID": anno_id,
            "ANNOTATION_REF": anno_ref,
            "PREVIOUS_ANNOTATION": "a" + str(last_anno -1),
        },
    )
    ET.SubElement(alignable_annotation, "ANNOTATION_VALUE").text = anno_value

    return new_annotation_element, alignable_annotation


def remove_first_previous_anno(new_annotation_element, i):
    """Remove the 'PREVIOUS_ANNOTATION' attribute for the first annotation."""
    if i == 0:
        new_annotation_element[0].attrib.pop("PREVIOUS_ANNOTATION", None)
    return new_annotation_element


def remove_any_previous_anno(new_annotation_element, i):
    """Remove the 'PREVIOUS_ANNOTATION' attribute from annotation."""
    new_annotation_element[0].attrib.pop("PREVIOUS_ANNOTATION", None)
    return new_annotation_element

                   
def handle_annos(anno, last_anno, root, split_pattern_all):
    """ look for annotations to split and then for each create children in other tiers
        replaces target anno with x split annos
        replaces chidren of target with x made annos"""
        
    
    for tier in root.findall("TIER"):
        tier_type = tier.get("LINGUISTIC_TYPE_REF", "")
        
        split_pattern = get_split_pattern(tier_type, split_pattern_all)
        
        
        for annotation in tier.findall("ANNOTATION"):
            ref_annotations = annotation.findall("REF_ANNOTATION")
            if not ref_annotations:
                continue

            ref_annotation = ref_annotations[0]
            annotation_id, annotation_ref, annotation_previous, annotation_value, annotation_previous = get_anno_info(ref_annotation)
            
            if annotation_id == anno:
                last_anno, anno_ids = add_split_pattern(ref_annotation, last_anno, tier, split_pattern)
                tier.remove(annotation)

            elif annotation_ref == anno:
                for i, new_value in enumerate(split_pattern):
                    last_anno += 1
                    anno_id = f'a{last_anno}'
                    anno_ref = anno_ids[i]
                    
                    
                    
                    new_annotation_element, alignable_annotation = make_anno(
                        tier, last_anno, anno_id, anno_ref, annotation_id, new_value)
                    
                    #remove_first_previous_anno(new_annotation_element, i)
                    remove_any_previous_anno(new_annotation_element, i)
                    
                    
                print(f"{annotation_value} --> {'|'.join(split_pattern)}")
                tier.remove(annotation)
            
            
            elif annotation_previous == anno:
                # update "Previous Annotation attrib" for following annotations
                ref_annotation.attrib["PREVIOUS_ANNOTATION"] = f'a{last_anno}'
                #print (annotation_value)
                #print (annotation_previous)
                
    print (" ")
    
    return last_anno


def split_annotations(change_list, file_in, file_out, split_pattern_all):
    
    tree = ET.parse(file_in)
    root = tree.getroot()
    
    last_anno = get_all_anno_IDs(root)[-1]

    for anno in change_list:
        
        last_anno = handle_annos(anno, last_anno, root, split_pattern_all)
                    
    ET.indent(tree, space="\t", level=0)
    tree.write(file_out, encoding="UTF-8", xml_declaration=True)

def find_mode(partners):
    #print (change_list)
    for group in partners:
        out_print = []
        for each in group:
            out_print.append("  " + each["TIER_TYPE"] + ": \t\t\t\t" +  each["ANNOTATION_VALUE"])

        print ("\n".join(out_print))
        print ("----------------------")

        
def find_all_matches(data, key2, word2, key3, word3):
    """ check if the strings are matching for TIER_TYPE and ANNOTATION_VALUE"""
    
    match2 = []
    match3 = []

    for k, v in data.items():

        match_check2 = check_if_match(v, key2, word2)
        match_check3 = check_if_match(v, key3, word3)

        if match_check2:
            match2.append(v)

        if match_check3:
            match3.append(v)
            
    return match2, match3

def update_UI(all_child_tiers):
    """ Fills in the depending tier types """
    if len(all_child_tiers) > 0:        
        key4_var.set(all_child_tiers[0])
        
    if len(all_child_tiers) > 1:
        key5_var.set(all_child_tiers[1])
    
    if len(all_child_tiers) > 2:
        key6_var.set(all_child_tiers[2])
        
    #print (all_child_tiers)

def process_files(mode):
    
    output_text.delete("1.0", "end")
    
    input_folder = input_path_var.get()
    output_folder = output_folder_var.get()
    
    key2 = key2_var.get()
    word2 = lexical_unit1_var.get()
    
    key3 = key3_var.get()
    word3 = pos1_var.get()
    replacer3 = pos2_var.get()
    
    key4 = key4_var.get()
    #word4 = pos3_var.get()
    replacer4 = pos4_var.get()
    
    key5 = key5_var.get()
    #word5 = pos4_var.get()
    replacer5 = pos6_var.get()
    
    key6 = key6_var.get()
    #word6 = pos5_var.get()
    replacer6 = pos7_var.get()
    
    split_pattern = generate_split_pattern(key3, word3, replacer3, 
                                           key4, replacer4,
                                           key5, replacer5,
                                           key6, replacer6)
    
    print (split_pattern)
    all_files = get_all_files(input_folder, output_folder)
    
    all_child_tiers = [] # to list all depending tiers when click on FIND
    
    
    for file_in, file_out in all_files:
        
        print ("\n", " ", file_in, "\n")
        
        data = get_data(file_in)

        data = find_children(data)
            
        match2, match3 = find_all_matches(data, key2, word2, key3, word3)
        
        change_list = [] 
        partners = []  # this is for the print out only partners are "tier_type : annotation_value"

        # bottom two rows are filled
        if (len(key2 + word2) > 0):
            change_list, partners = two_bottom_rows_filled(match2, match3)

        else:
            change_list, partners = one_rows_filled(match3)
        
        
        #print (change_list)
        for each in print_child_list(change_list, data):
            if each not in all_child_tiers:
                all_child_tiers.append(each)
        
        if mode == "find_only":
            find_mode(partners)
            
            
     
        if mode == "split":
            split_annotations(change_list, file_in, file_out, split_pattern)
    
    
    update_UI(all_child_tiers)
    
    print ("++++++++++++++++++++++++++++++++++++")    
    print (" DEPENDING TIERS: " + "  ".join(all_child_tiers))
    print('+++ DONE +++')  
                 
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++        
# USER INTERFACE


def print_instructions():
    
    output_text.delete("1.0", "end")
    
    print (""" 

 INSTRUCTIONS:
 
 There are two overlapping functionalities, Find (step 1) and Split (step 2)
 
 
    1.
    ╭――――――――――――――――――╮
    │ [    ]    [    ] │
   ╭┼――――――――――――――――――┼――――――――――╮
   ││ [    ]    [  x ] │  [    ]  │ 2. 
   │╰――――――――――――――――――╯          │
   │ [    ]               [    ]  │
   │                              │
   │ [    ]               [    ]  │ 
   ╰――――――――――――――――――――――――――――――╯ 
                      Find         Split
                      
 
 
 1. FIND THE ANNOTATION YOU WANT TO CHANGE
 
    Use FIND to get a list of annotations you want to change using the 4 cells in the top left. 
    
    You can specify the parent to get better results for the target. You ca use regular expressions.
 

    Tier type:                  Look for:
 
    parent: [       ]           [            ]
 
    [               ]           [   target   ]
    
 
    When you click FIND, all matches for the target are listed, at the bottom you will also 
    see the DEPENDING TIERS. 
    
    The Tier Type column shows any depending tiers.
    
    ALL DEPENDING TIERS have to be handled in the next step !
    
  
  
 
 2. SPLIT THE ANNOTATIONS
 
    If the target annotation you want to split has DEPENDING TIERS, they also need to get split.
    
    Use the bottom three rows to identify the tier type, and what you 
    want it to be split into.
    
    Use '|' to separate the new annotation.
    
    You need to specify the target tier and all depending 'child tiers'.
 
    
    Tier type:                  Look for:                  Replace with:
    
    [  target  tier  ]          [     old       ]          [     new|new   ]
    
    [  depent. tier1 ]                                     [     new|new   ]
    
    [  depent. tier2 ]                                     [     new|new   ]
    
    
    
    When you click on Split, the annotations will be split based on 
    the 'Replace with' column. 
    
    The files will be saved to the output folder.
 
""")

        
# Redirect standard output and error to the Text widget
class TextRedirector:
    def __init__(self, text_widget):
        self.text_widget = text_widget
    
    def write(self, text):   
        self.text_widget.insert("end", text)
        
    def flush(self):  # Needed for some interactive environments
        pass  

# USER INTERFACE ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Main Tkinter window 
root = tk.Tk()
root.title("EAF ANNOTATION SPLITTER")

# Variables
current_dir = os.getcwd()

input_path_var = tk.StringVar(value=current_dir)
output_folder_var = tk.StringVar(value=current_dir)

key2_var = tk.StringVar(value="words")
key3_var = tk.StringVar(value="lexical-unit")
key4_var = tk.StringVar(value="")
key5_var = tk.StringVar(value="")
key6_var = tk.StringVar(value="")

lexical_unit1_var = tk.StringVar(value=".*")

pos1_var = tk.StringVar(value="saipid")

pos2_var = tk.StringVar(value="sa|i-|pid")
pos4_var = tk.StringVar(value="ascend|3SG.S|DWN")
pos6_var = tk.StringVar(value="Verb|v.pref|Verb")
pos7_var = tk.StringVar(value="test|test|test")

split_var = tk.StringVar()


# Input path selection
tk.Label(root, text="Input Folder:").grid(row=0, column=0, sticky="e", padx=5, pady=5)
tk.Entry(root, textvariable=input_path_var, width=100).grid(row=0, column=1, padx=5, pady=5, sticky="e")
tk.Button(root, text="Browse", width=15, command=lambda: select_folder(input_path_var)).grid(row=0, column=2, padx=5, pady=5)


# Output folder selection
tk.Label(root, text="Output Folder:").grid(row=1, column=0, sticky="e", padx=5, pady=5)
tk.Entry(root, textvariable=output_folder_var, width=100).grid(row=1, column=1, padx=5, pady=5, sticky="e")
tk.Button(root, text="Browse", width=15, command=lambda: select_folder(output_folder_var)).grid(row=1, column=2, padx=5, pady=5)


# Search section
tk.Label(root, text="    Tier Type:").grid(row=2, column=0, sticky="w", padx=5, pady=5)
tier_type_var = tk.StringVar()  # Variable to hold the tier type input
tk.Entry(root, textvariable=tier_type_var, width=20).grid(row=2, column=0, padx=5, pady=5, sticky="e")

search_label = "                                                                                           Search String:"
tk.Label(root, text=search_label).grid(row=2, column=1, sticky="w", padx=5, pady=5)
search_string_var = tk.StringVar()  # Variable to hold the search string
tk.Entry(root, textvariable=search_string_var, width=40).grid(row=2, column=1, padx=5, pady=5, sticky="e")
tk.Button(root, text="Search", width=15, command=lambda: find_string(input_path_var.get(), 
                                                           {tier_type_var.get(): search_string_var.get()})).grid(row=2, column=2, padx=5, pady=5)


# Horizontal line below row 2 ---------------------------------------------------------------------------------------
separator = ttk.Separator(root, orient='horizontal')
separator.grid(row=3, column=0, columnspan=3, sticky="ew", pady=5)

tk.Label(root, text=" SPLIT ANNOTATIONS AND DEPENDENTS").grid(row=4, column=0, sticky="w", columnspan=3, padx=5, pady=5)

tk.Button(root, text="Instructions",  bg='white', width=15, command=lambda: print_instructions()).grid(row=4, column=2, sticky="e", padx=10, pady=10)

# Labels for columns
tk.Label(root, text=" Tier Type:").grid(row=5, column=0, sticky="w", padx=5, pady=5)
tk.Label(root, text="Look for:").grid(row=5, column=1, sticky="w", padx=5, pady=5)
#tk.Label(root, text="Replace with:").grid(row=5, column=1, sticky="e", padx=5, pady=5)

# second row of data
tk.Entry(root, textvariable=key2_var, width=35).grid(row=6, column=0, padx=5, pady=5)
tk.Entry(root, textvariable=lexical_unit1_var, width=35).grid(row=6, column=1, padx=5, pady=5, sticky="w")
#tk.Label(root, text="parent: ").grid(row=6, column=0, sticky="w", padx=5, pady=5)
tk.Label(root, text="Replace with:                                              ").grid(row=6, column=1, sticky="e", padx=5, pady=5)


# third row of data
tk.Entry(root, textvariable=key3_var, width=35).grid(row=7, column=0, padx=5, pady=5)

frame2 = tk.Frame(root, bg="orange", padx=0, pady=0)
frame2.grid(row=7, column=1, padx=0, pady=0, sticky="w")

tk.Entry(frame2, textvariable=pos1_var, width=35, bg='papaya whip').grid(row=7, column=1, padx=5, pady=5, sticky="w")
tk.Entry(root, textvariable=pos2_var, width=35, bg='papaya whip').grid(row=7, column=1, padx=5, pady=5, sticky="e")

# fourth row of data
tk.Entry(root, textvariable=key4_var, width=35).grid(row=8, column=0, padx=5, pady=5)
#tk.Entry(root, textvariable=pos3_var, width=35, bg='yellow').grid(row=8, column=1, padx=5, pady=5, sticky="w")
tk.Entry(root, textvariable=pos4_var, width=35, bg='yellow').grid(row=8, column=1, padx=5, pady=5, sticky="e")

# fifth row of data
tk.Entry(root, textvariable=key5_var, width=35).grid(row=9, column=0, padx=5, pady=5)
#tk.Entry(root, textvariable=pos5_var, width=35, bg='yellow').grid(row=9, column=1, padx=5, pady=5, sticky="w")
tk.Entry(root, textvariable=pos6_var, width=35, bg='yellow').grid(row=9, column=1, padx=5, pady=5, sticky="e")

# sixth row of data
tk.Entry(root, textvariable=key6_var, width=35).grid(row=10, column=0, padx=5, pady=5)
#tk.Entry(root, textvariable=pos5_var, width=35, bg='yellow').grid(row=10, column=1, padx=5, pady=5, sticky="w")
tk.Entry(root, textvariable=pos7_var, width=35, bg='yellow').grid(row=10, column=1, padx=5, pady=5, sticky="e")

# Process button
tk.Button(root, text="Find",  bg='orange', width=15, command=lambda: process_files("find_only")).grid(row=11, column=0, columnspan=3, pady=10)
tk.Button(root, text="Split",  bg='red', width=15, command=lambda: process_files('split')).grid(row=11, column=2, columnspan=3, pady=10)

# Horizontal line below row 8 -----------------------------------------------------------------------------------------
separator2 = ttk.Separator(root, orient='horizontal')
separator2.grid(row=12, column=0, columnspan=3, sticky="ew", pady=5)

root.grid_rowconfigure(12, weight=1)  # Allow row 9 to grow/shrink
# Output Textarea
output_text = st.ScrolledText(root, height=22, width=120, wrap="word")
output_text.grid(row=13, column=0, columnspan=3, padx=5, pady=5, sticky="nsew")
output_text.configure(state="normal")

# Redirect stdout and stderr to the Text widget
sys.stdout = TextRedirector(output_text)
sys.stderr = TextRedirector(output_text)

# Run the Tkinter event loop
root.mainloop() 

# Sandbox 02